In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook
import json
import os
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt

utils = Utils()
import time

2023-12-02 06:30:07.676587: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 06:30:07.676619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 06:30:07.677688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 06:30:07.684292: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 06:30:08.516893: W tensorflow/comp

In [3]:
def create_dataframe(experiment):
    datagenerator = DataGenerator()

    noise_type = (
        None if experiment["noise_type"] == "normal" else experiment["noise_type"]
    )  # irrelevant,duplicated
    anomaly_type = (
        None if experiment["anomaly_type"] == "normal" else experiment["anomaly_type"]
    )  # cluster,global,local,dependency

    datagenerator.dataset = experiment["dataset"]
    data = datagenerator.generator(
        la=1.00,
        realistic_synthetic_mode=anomaly_type,
        noise_type=noise_type,
        noise_ratio=float(experiment["noise_ratio"]),
        stdscale=True,
        minmax=False,
    )

    Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
    Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
    lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
    lb_test = torch.tensor(data["y_test"], dtype=torch.float32)

    return Y_train, Y_test, lb_train, lb_test

In [4]:
EXPERIMENTS_FILE = "experiments/complete/000_datasets_01_47_normal.json"
with open(EXPERIMENTS_FILE) as file:
    experiments = json.load(file)

In [5]:
# (
#    df.replace({"anomaly_type": "normal"}, "cluster").to_json(
#        "experiments/complete/001_datasets_01_to_06_cluster.json", orient="records"
#    )
# )

In [ ]:
success_experiments = []
failed_experiments = []
for experiment in tqdm_notebook(experiments):
    Y_train, Y_test, lb_train, lb_test = create_dataframe(experiment)
    Y_val, Y_test, lb_val, lb_test = train_test_split(
        Y_test, lb_test, test_size=0.50, random_state=42
    )

    if experiment["loss"] == "normal":
        idx_n = np.where(lb_train == 0)[0]
        Y_train_normal = Y_train[idx_n]
        lb_train_normal = lb_train[idx_n]

    n_train = len(Y_train_normal)
    data_dim = Y_train_normal.shape[1]
    kernel = experiment["kernel"]
    latent_dim = int(experiment["latent_dim"])
    nn_layers = tuple(map(int, experiment["layers"].split(",")))
    n_inducing = int(experiment["n_inducing"])
    n_epochs = int(experiment["n_epochs"])
    lr = float(experiment["learning_rate"])
    batch_size = int(experiment["batch_size"])

    # print(experiment)

    try:
        gplvm = AD_GPLVM(
            latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size, kernel
        )

        # Fitting the Model
        train_start_time = time.time()
        gplvm.fit(Y_train_normal)
        train_end_time = time.time()

        # Validating the Model
        val = []
        for i in range(100):
            score = gplvm.predict_score(Y_val)
            val.append(score)
        validation_score = np.mean(val, axis=0)
        # validation_score = gplvm.calculate_train_elbo(Y_train_normal)

        # Results
        pred_start_time = time.time()
        test_score = gplvm.predict_score(Y_test)
        pred_end_time = time.time()

        # Save Metrics
        metrics = utils.metric(y_true=lb_val, y_score=validation_score)
        validation_metrics = utils.metric(y_true=lb_test, y_score=test_score)
        experiment["negative_elbo"] = validation_score.sum()  # validation_score.sum()
        experiment["train_loss_curve"] = gplvm.loss_list
        experiment["val_auc_roc"] = validation_metrics["aucroc"]
        experiment["val_auc_pr"] = validation_metrics["aucpr"]
        experiment["test_auc_roc"] = metrics["aucroc"]
        experiment["test_auc_pr"] = metrics["aucpr"]
        experiment["training_time"] = train_end_time - train_start_time
        experiment["inference_time"] = pred_end_time - pred_start_time

        # Reconstrucao
        Y_val_recon, Y_val_recon_covar = gplvm.model.reconstruct_y(Y_val)
        experiment["val_reconstruct_error"] = float(utils.rmse(Y_val, Y_val_recon.T))

        utils.save_experiment(experiment)

    except Exception as error:
        print("An exception occurred:", error)
        experiment["test_auc_roc"] = 0.0
        experiment["test_auc_pr"] = 0.0
        utils.save_experiment(experiment)

/tmp/ipykernel_16521/1405413897.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for experiment in tqdm_notebook(experiments):


  0%|          | 0/3384 [00:00<?, ?it/s]

subsampling for dataset 01_ALOI...


/home/ec2-user/SageMaker/OE-GPLVM/new_aeb_gplvm.py:319: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1686274778240/work/aten/src/ATen/native/TensorShape.cpp:3571.)
  klu_expanded = ll_shape.T.add_(klu).sum(-1).T.div((self.n_train))


subsampling for dataset 01_ALOI...
subsampling for dataset 01_ALOI...
subsampling for dataset 01_ALOI...
subsampling for dataset 32_shuttle...
subsampling for dataset 32_shuttle...
subsampling for dataset 32_shuttle...
subsampling for dataset 32_shuttle...


In [ ]:
# pd.DataFrame(success_experiments).to_json("experiments/kernels/001_matern_rbf_all_results.json", orient="records")

In [ ]:
# success_experiments

In [ ]:
df_multi = pd.DataFrame(success_experiments)
df_multi.head(3)

In [ ]:
df_multi.columns

In [ ]:
x, y = "test_auc_roc", "val_auc_pr"
filter_df = df_multi.dataset == "29_Pima"
plt.scatter(df_multi[filter_df][x], df_multi[filter_df][y])
plt.xlabel(x)
plt.ylabel(y)
plt.title(pearsonr(df_multi[filter_df][x], df_multi[filter_df][y]).statistic)
None